In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode

import nltk
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans, DBSCAN

sns.set()
sns.set_style('darkgrid')
sns.set_palette('deep')


nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
data = pd.read_csv('nfedataEAN.csv', low_memory=False)

In [ ]:
data.info()

In [ ]:
data['produto_ean_dbscan'] = np.zeros((data.shape[0],))

In [ ]:
data['produto_codigo_ean'] = data['produto_codigo_ean'].apply(str)

In [ ]:
for gtin in data['produto_codigo_ean'].unique():
    mask = data['produto_codigo_ean'] == gtin
    # pegar descrições
    desc = data.produto_descricao.loc[mask]
    if desc.shape[0] <= 5:
        continue 
    desc = desc.apply(lambda x: str(x).lower())
    desc = desc.apply(unidecode)
    desc = desc.apply(lambda x: ' '.join(
        [
            w for w in word_tokenize(x) 
            if w not in stopwords.words('portuguese')
            ]
        )
    )
    desc = desc.apply(lambda x: re.sub(r'[0-9]{3,}', '', x))
    cv = CountVectorizer(ngram_range=(1, 3))
    desc_tf = cv.fit_transform(desc)
    desc_tf = pd.DataFrame(desc_tf.toarray(), columns=)
    
    n_components = 3
    if n_components >= desc_tf.shape[1]:
        desc_pca = desc_tf
    else:
        pca = TruncatedSVD(n_components=n_components)
        desc_pca = pca.fit_transform(desc_tf)
    normalizer = Normalizer(copy=False, norm='l2')
    desc_pca = normalizer.fit_transform(desc_pca)
    df = pd.DataFrame(desc_pca)

    dbs = DBSCAN(eps=0.3, metric='cosine')
    desc_db = dbs.fit_predict(desc_pca)
    desc_db[desc_db != 0] = -1
    data.loc[mask, 'produto_ean_dbscan'] = desc_db

In [ ]:
data['produto_ean_dbscan'].value_counts()

In [ ]:
data.to_csv('nfedataDBSCAN.csv', index=False)